'2wQgqMaqTz6EIKjGqt8-QA'

In [66]:
bike_road = pd.read_csv('model_1/total_data.csv')
bike_road[(bike_road['월']==12)&(bike_road['행정구']=='강남구')]['자전거전용도로km'].max()

114.0

In [85]:
def predict_demand(part='강남구',date='2025-06-21') :
    # 필요 라이브러리 호출
    from tensorflow.keras.models import load_model
    import numpy as np
    import dotenv
    import os
    import time
    import datetime
    import pandas as pd
    import matplotlib.pyplot as plt
    import requests
    import xml.etree.ElementTree as ET
    import requests
    import warnings
    from bs4 import BeautifulSoup
    from sklearn.preprocessing import LabelEncoder, StandardScaler
    
    dotenv.load_dotenv()
    API_KEY = os.getenv('weather')
    API_KEY
    
    model = load_model('model_1/DNN_model_1.h5')
    # 경고창 무시
    warnings.simplefilter("ignore")

    dotenv.load_dotenv() # API 사용을 위한 인증키 로드
    API_KEY = os.getenv('SEOUL_DATA')

    # API를 통해 불러올 주요 장소의 이름과 카테고리 체크
    inverse = pd.read_csv('data/역별행정구.csv')
    spot = inverse[inverse['행정구']==part]['지역명'].iloc[0]
    
    date='2025-06-21'
    date_list = date.split('-')
    year = int(date_list[0]); month = int(date_list[1]); day =int(date_list[2])
    weekday = datetime.date(int(date_list[0]),int(date_list[1]),int(date_list[2])).weekday()
    is_weekend = 1 if weekday>=5 else 0
    
    bike_road = pd.read_csv('model_1/total_data.csv')
    bike_road_len = bike_road[(bike_road['월']==12)&(bike_road['행정구']==part)]['자전거전용도로km'].max()
    
    df = pd.read_csv("model_1/total_data.csv", parse_dates=["일시"])
    le_gu   = LabelEncoder()
    df["gu_le"] = le_gu.fit_transform(df["행정구"])
    df["month"]      = df["일시"].dt.month
    df["weekday"]    = df["일시"].dt.weekday
    df["is_weekend"] = (df["weekday"]>=5).astype(int)
    bins   = [0, 1500, 3000, 4500, 6000,7500,9000, np.inf]
    labels = ["0–1499", "1500–2999","3000–4499", "4500–5999", "6000–7499", "7500-8999","9000+"]

    df["y_bin"] = pd.cut(df["총대여량"], bins=bins, labels=labels, right=False)
    df.dropna(subset=["y_bin"], inplace=True)
    
    le_y = LabelEncoder()
    df["y_label"] = le_y.fit(df["y_bin"])
    n_classes = df["y_label"].nunique()
    
    FEATURE_WEATHER = ["평균기온(°C)","일강수량(mm)","평균 풍속(m/s)","평균 상대습도(%)"]
    FEATURES = ["gu_le","month","weekday","is_weekend",'총생활인구수','자전거전용도로km'] + FEATURE_WEATHER
    
    train_df = df[df["일시"].dt.year != 2024]
    X_tr = train_df[FEATURES].values

    scaler = StandardScaler().fit(X_tr)
    try:
        url = f"http://openapi.seoul.go.kr:8088/{API_KEY}/xml/citydata/1/5/{spot}"
        response = requests.get(url, timeout=10)
        soup = BeautifulSoup(response.text, features='html.parser')

        data = {
            '유동인구': soup.select_one("AREA_PPLTN_MAX").text,
            '습도': soup.select_one("HUMIDITY").text,
            '기온': soup.select_one("TEMP").text,
            '장소이름': spot,
            '강수량' : soup.select_one("PRECIPITATION").text,
            '풍속' : soup.select_one("WIND_SPD").text,
        }

    except Exception as e:
        print(f"[{spot}] 수집 실패: {e}")
        model = load_model('model_1/DNN_model_1.h5')
        
    sample = pd.DataFrame([{
            "gu_le":        le_gu.transform([part])[0],
            "month":        month,
            "weekday":      weekday,
            "is_weekend":   is_weekend,
            "평균기온(°C)":  data.get('기온',15),
            "일강수량(mm)":  data.get('강수량',15) if isinstance(data.get('강수량',15),float) else 0,
            "평균 풍속(m/s)": data.get('풍속',2),
            "평균 상대습도(%)":data.get('습도',50),
            '총생활인구수' : data.get('유동인구',25000)*12,
            '자전거전용도로km' : bike_road_len 
        }])
    X_samp = scaler.transform(sample[FEATURES].values.reshape(1,-1))

        # DNN 예측
    bin_idx_dnn = model.predict(X_samp).argmax(axis=1)[0]
    print("▶ DNN 예측 구간:", le_y.inverse_transform([bin_idx_dnn])[0])
    return(le_y.inverse_transform([bin_idx_dnn])[0])

In [86]:
predict_demand('강남구','2025-06-22')

ParserError: Error tokenizing data. C error: out of memory